In [1]:
with open("input.txt", 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [3]:
stoi = { ch: i for i,ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s : [stoi[c] for c in s]
decode = lambda i: ''.join([itos[c] for c in i])

print(encode("Whats up?"))
print(decode(encode("Whats yo?")))

[35, 46, 39, 58, 57, 1, 59, 54, 12]
Whats yo?


In [4]:
# import tiktoken
# enc = tiktoken.get_encoding("o200k_base")

In [5]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data[:100])

torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [6]:
n = int(0.9*len(data))
train_data = data[:n]
val_data= data[n:]

print(train_data.shape, val_data.shape)

torch.Size([1003854]) torch.Size([111540])


In [ ]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]

for i in range(block_size):
    context = x[:i+1]
    target = y[i]
    print(context, target)



tensor([18]) tensor(47)
tensor([18, 47]) tensor(56)
tensor([18, 47, 56]) tensor(57)
tensor([18, 47, 56, 57]) tensor(58)
tensor([18, 47, 56, 57, 58]) tensor(1)
tensor([18, 47, 56, 57, 58,  1]) tensor(15)
tensor([18, 47, 56, 57, 58,  1, 15]) tensor(47)
tensor([18, 47, 56, 57, 58,  1, 15, 47]) tensor(58)


In [9]:
torch.manual_seed(1337)

batch_size = 4
block_size = 8

def get_batch(split):
    
    data = train_data if split=='train' else val_data
    
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb,yb = get_batch('train')
print(xb.shape,xb)
print(yb.shape, yb)

for i in range(batch_size):
    for j in range(block_size):
        _xb = xb[i,:j+1]
        _yb = yb[i,j]
        print(f"Input: {_xb} Output: {_yb}")
        

torch.Size([4, 8]) tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
torch.Size([4, 8]) tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
Input: tensor([24]) Output: 43
Input: tensor([24, 43]) Output: 58
Input: tensor([24, 43, 58]) Output: 5
Input: tensor([24, 43, 58,  5]) Output: 57
Input: tensor([24, 43, 58,  5, 57]) Output: 1
Input: tensor([24, 43, 58,  5, 57,  1]) Output: 46
Input: tensor([24, 43, 58,  5, 57,  1, 46]) Output: 43
Input: tensor([24, 43, 58,  5, 57,  1, 46, 43]) Output: 39
Input: tensor([44]) Output: 53
Input: tensor([44, 53]) Output: 56
Input: tensor([44, 53, 56]) Output: 1
Input: tensor([44, 53, 56,  1]) Output: 58
Input: tensor([44, 53, 56,  1, 58]) Output: 46
Input: tensor([44, 53, 56,  1, 58, 46]) Output: 39
Input: tensor([44, 53

In [21]:
import torch 
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

    
    def forward(self, idx, targets):
        logits = self.token_embedding_table(idx)
        
        B, T, C = logits.shape
        
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, target=targets)
        return logits, loss

model = BigramLanguageModel(vocab_size=vocab_size)
logits, loss = model(xb,yb)
print(loss)

tensor(4.8786, grad_fn=<NllLossBackward0>)
